#### Prototype FuzzyPandas Functions using U.S. News and IPEDS Data

In [ ]:
import time
import functools

import numpy as np
import pandas as pd

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

**First, let's read in the school ranking data scraped from usnews.com & do a little pre-processing**

In [ ]:
def read_us_news(pickle):
    df = pd.read_pickle(pickle)   
    return pd.concat([us_news, df], axis=0, ignore_index=True)

us_news = pd.DataFrame() #initialize empty data frame
us_news = read_us_news("example_data/us_news/usnews-ranking-national-universities.pickle")
us_news = read_us_news("example_data/us_news/usnews-ranking-national-liberal-arts-colleges.pickle")
us_news = read_us_news("example_data/us_news/usnews-ranking-regional-colleges-midwest.pickle")
us_news = read_us_news("example_data/us_news/usnews-ranking-regional-colleges-north.pickle")
us_news = read_us_news("example_data/us_news/usnews-ranking-regional-colleges-south.pickle")
us_news = read_us_news("example_data/us_news/usnews-ranking-regional-colleges-west.pickle")
us_news = read_us_news("example_data/us_news/usnews-ranking-regional-universities-midwest.pickle")
us_news = read_us_news("example_data/us_news/usnews-ranking-regional-universities-north.pickle")
us_news = read_us_news("example_data/us_news/usnews-ranking-regional-universities-south.pickle")
us_news = read_us_news("example_data/us_news/usnews-ranking-regional-universities-west.pickle")

us_news["city"] = us_news.location.apply(lambda x: x.split(", ")[0].upper().strip()) #parse out city, to upper case
us_news["state"] = us_news.location.apply(lambda x: x.split(", ")[1].upper().strip()) #parse out state, to upper case
us_news["school"] = us_news.school.apply(lambda x: x.upper().strip()) #school name to upper case

us_news.drop("location", axis=1, inplace=True) #drop original location (now that we've split out into city and state)
us_news.head(5)

**Read in IPEDS Data & Preprocess. IPEDS Data can be downloaded for free here:**
https://nces.ed.gov/ipeds/datacenter/Default.aspx

In [ ]:
ipeds=pd.read_csv("example_data/ipeds/HD2014.csv")[["UNITID","INSTNM","CITY","STABBR"]]

ipeds.rename(columns={"UNITID":"unitid","INSTNM":"school","CITY":"city","STABBR":"state"},inplace=True) 
#rename columns to match U.S. News Data

ipeds["school"] = ipeds.school.apply(lambda x: x.upper().strip()) #school to upper case
ipeds["city"] = ipeds.city.apply(lambda x: x.upper().strip()) #city to upper case
ipeds["state"] = ipeds.state.apply(lambda x: x.upper().strip()) #state to upper case
ipeds.head(5)

** Subset Data to Only NY and CT Schools for Testing**

In [ ]:
test_us_news = us_news[us_news.state.isin(["CT","NY"])].copy()
test_ipeds = ipeds[ipeds.state.isin(["CT","NY"])].copy()

print us_news.shape, test_us_news.shape
print ipeds.shape, test_ipeds.shape

#### Single Key Fuzzy Match

In [ ]:
def decode_column(text):
    if type(text) == unicode:
        return text.encode("ascii", "replace")
    else:
        return text
        
def single_key_match(dfA, dfB, key, scorer=fuzz.ratio, cutoff=0.6): #Use force key for later
    start = time.time()
    
    rowsA[key] = dfA[key].apply(decode_text) #If column is unicode, convert to string
    rowsA[key] = dfB[key].apply(decode_text) #if column is unicode, convert to string
    
    rowsA = rowsA.drop_duplicates(subset=key)[key].T.to_dict().values() #Convert DF -> List of Dicts
    rowsA = rowsB.drop_duplicates(subset=key)[key].T.to_dict().values() #Convert DF -> List of Dicts
    
    matches = pd.DataFrame() #Initialize empty dataframe to hold matches
    
    for rowA in rowsA:
        max_score = 0
        
        for rowB in rowsB:                
            score = scorer(rowA, rowB)
            
            if score > cutoff and score > max_score:
                match = {key+"A": rowA, key+"B": rowB, key+"_match_score": score}
                max_score = score
            
        if match is None:
            match = {key+"A": keyA, key+"B": np.nan, key+"_match_score": np.nan}
            
        matches = matches.append([match], ignore_index=True)
                              
    end = time.time()
    print "Duration: ", round(end-start, 2), "Seconds"

    return matches

In [ ]:
matches = match_single_key(test_us_news, test_ipeds, key="school", cutoff=0.6)

#### Multi Key Fuzzy Match (hierarchical) -- NEED TO FINISH WRITING THIS FUNCTION

In [ ]:
def multi_key_match(dfA, dfB, keys, scorer=fuzz.ratio, cutoffs=None):
    start = time.time()
    
    if cutoffs is None:
        cutoffs=[0.6] * len(keys) #Default to 0.6 for each key
        
    rowsA[key] = dfA[key].apply(decode_text) #If column is unicode, convert to string
    rowsB[key] = dfB[key].apply(decode_text) #if column is unicode, convert to string
        
    rowsA["filter"] = " " #Initialize Filter on DF A
    rowsB["filter"] = " " #Initialize Filter on DF B
    
    rowsA = rowsA.drop_duplicates(subset=keys)[keys+["filter"]].T.to_dict().values() #Convert DF -> List of Dicts
    rowsB = rowsB.drop_duplicates(subset=keys)[keys+["filter"]].T.to_dict().values() #Convert DF -> List of Dicts    
    
    for key, cutoff in zip(keys, cutoffs): #Iterate through key variables (should be ordered most general -> specific)  
        for rowA in [x for x in rowsA[key]]: #check list comprehension syntax
            max_score = 0
            
            for rowB in [x for x in rowsB[key] if #FILTERING#]
                score = scorer(rowA, rowB)
            
            if score > cutoff and score > max_score:
                match = {key+"A": rowA, key+"B": rowB, key+"_match_score": score}
                max_score = score
            
            if match is None:
                match = {key+"A": rowA, key+"B": np.nan, key+"_match_score": np.nan}
        
            matches = matches.append([match], ignore_index=True)
            matches = pd.merge() #Match and set filter
            
        rowA
                              
    end = time.time()
    print "Duration: ", round(end-start, 2), "Seconds"

    return matches
        
##Function to Crate Crosswalk - OLD CODE TO INCORPORATE
def create_xwalk(a, b, byvars, cutoffs, scorer):
    b["filter"] = b[["constant"]+byvars].apply(lambda x: "#".join(x), axis=1).str.split("#")
    
    a = a.drop_duplicates(subset=byvars)[byvars].T.to_dict().values()
    b = b.drop_duplicates(subset=byvars)[byvars+["filter"]].T.to_dict().values()
    
    for row in a:
        for i in range(0, len(byvars)):
            byvar = byvars[i]
            cutoff = cutoffs[i]
            if i==0:
                choices = [x[byvar] for x in b]
            else:
                choices = [x[byvar] for x in b if row["filter"]==x["filter"][:i]]
            
            row["matched_"+byvar] = fuzzy_match(row[byvar], choices, cutoff=cutoff, scorer=scorer)[1]
            row["filter"] = row["filter"]+[row["matched_"+byvar]]
            
    return pd.DataFrame(a).drop(["constant","filter"], axis=1)

In [ ]:
matches = multi_key_match(test_us_news, test_ipeds, keys=["state","city","school"])
matches.head(5)

#### Use Fuzzy Matching to Join Two Pandas Dataframes

In [ ]:
def fuzzy_merge(dfA, dfB, key, how="left", scorer=fuzz.ratio, cutoff=None, force_single=False):
    dfA["inA"] = 1 #Flag rows existing in DF A
    dfB["inB"] = 1 #Flag rows existing in DF A
    
    merged = pd.merge(a, b, on=key, how="outer") #Perform a regular (exact) join
    
    matched = merged[(merged.in_a.notnull()) & (merged.in_b.notnull())].copy() #Sift out exact matches
    nomatchA = merged[(merged.in_a.notnull() | (merged.in_b.isnull()))].copy() #Sift out nonmatches from DF A
    nomatchB = merged[(merged.in_a.isnull()) | (merged.in_b.notnull())].copy() #Sift out nomathches from DF B
    
    if nomatch_a.shape[0]==0 | nomatch_b.shape[0]==0: #No nomatches. Return matched DF and end.
        return matched
    
    elif len(key) == 1: #Fuzzy Matching - Single Key
        matches = 
    
    elif force_single == True: #Fuzzy Matching - Multi Key Non-Heirarchical
        pass
    
    else: #Fuzzy Matching - Multi Key Heirarchical
        pass

In [ ]:
def fuzzy_merge(a, b, key, cutoff=0.6, how="left", match_function=ratio_match):
    a["in_a"] = 1 #Flag rows existing in DF A
    b["in_b"] = 1 #Flag rows existing in DF B
    
    merged = pd.merge(a, b, on=key, how="outer") #Perform a regular (exact) join
    
    matched = merged[(merged.in_a.notnull()) & (merged.in_b.notnull())].copy() #Sift out exact matches
    nomatchA = merged[(merged.in_a.notnull() | (merged.in_b.isnull()))].copy() #Sift out nonmatches from DF A
    nomatchB = merged[(merged.in_a.isnull()) | (merged.in_b.notnull())].copy() #Sift out nomathches from DF B
    
    if nomatch_a.shape[0]==0 | nomatch_b.shape[0]==0: #If no nonmatches in A or B, return matches & end
        return matched

    else: #Otherwise, proceed to fuzzy matching
        crosswalk = 
        
        
    crosswalk = match_function(nonmatch_a, nonmatch_b, key=key, cutoff=cutoff) #Create key crosswalk
        
        fuzzy_merge1 = pd.merge(nonmatch_a, crosswalk, on=key+"_A", how="left") #Match DF A to crosswalk on Key 
        fuzzy_merge2 = pd.merge(fuzzy_merge1, nonmatch_b, on=key+"_B", how=how) #Match DF A+Xwalk to DF B on Key B

        return pd.concat([matched, fuzzy_merge_2], axis=0) #Append fuzzy matches to exact matches and return (MAKE DROPS)